In [ ]:
!pip install automatic_speech_recognition
# !pip install numpy
# !pip install pandas
# !pip install scipy
# !pip install sklearn

     |████████████████████████████████| 40 kB 2.6 MB/s 
  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=216c5035f5ba15c6078af9654647758f6fb7271e3043d3916553b44524213290
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the binning methods

# ------- Logorithmic Spacing Bins (Mel Scale) ----------
def getMelBins(numBins, highFreq):
    # formula for hertz to mel
    # m = 1127 * natural log(1 + f/700)
    # formula for mel to hertz
    # f = 700*(e^(m/1127) - 1)

    # Map biggest freq to mel scale
    # Do evenly spaced sampling along the mel scale
    # Map those back to hertz to get our mel-like log scale

    # Upper bound for mel
    stop = 1127 * np.log(1 + highFreq/700)
    # print("Upper Mel: ", stop)

    mels = np.linspace(0, stop, num=numBins + 1)
    # print("Mels: ", mels)
    
    # inverse of mel to hertz
    binEdges1 = 700*(np.exp(mels/1127) - 1)

    # Cast to integers
    binEdges1 = binEdges1.astype(int)

    # Convert to array of two tuples (each bin with its start and end)
    binEdges = []
    for i in range(len(binEdges1)-1):
        binEdges.append((binEdges1[i], binEdges1[i+1]))

    return binEdges

# call:
# binEdges = getMelBins(NUM_BINS, len(halfDFT) - 1)
# -------- Logorithmic Spacing Bins ----------


# -------- Evenly Spaced Bins --------
def getEvenBins(numBins, highFreq):
    binEdges1 = np.linspace(0, highFreq, num=numBins + 1).astype(int)

    # Convert to array of two tuples (each bin with its start and end)
    binEdges = []
    for i in range(len(binEdges1)-1):
        binEdges.append([binEdges1[i], binEdges1[i+1]])

    return binEdges

# call:
# binEdges = getEvenBins(NUM_BINS, len(halfDFT) - 1)
# -------- Evenly Spaced Bins --------


# -------- Overlapping Evenly Spaced Bins --------
def getOverlappingEvenBins(numBins, highFreq):
    nFirstBins = np.ceil(numBins/2).astype(int)
    # nSecondBins = nFirstBins - 1

    binEdges1 = getEvenBins(nFirstBins, highFreq)

    # create bins halfway between other bins
    binEdges2 = []
    for i in range(len(binEdges1) - 1):
        binEdges2.append([((binEdges1[i][0] + binEdges1[i][1])/2).astype(int), ((binEdges1[i+1][0] + binEdges1[i+1][1])/2).astype(int)])
    
    # combine into an array of two tuples (each bin with its start and end)
    binEdges = []
    for i in range(len(binEdges1)):
        binEdges.append(binEdges1[i])
        if i < len(binEdges2):
            binEdges.append(binEdges2[i])

    return binEdges

# call:
# binEdges = getOverlappingEvenBins(NUM_BINS, len(halfDFT) - 1)
# -------- Overlapping Evenly Spaced Bins --------


# -------- Bins Based on Peaks --------
def getPeakBins(halfDFT, DOWN_SAMPLE_FACTOR):
    filteredDft = np.convolve(np.abs(halfDFT), np.array([0.5] * 5))
    # DOWN_SAMPLE_FACTOR = 8
    DOWN_SAMPLE_FACTOR = DOWN_SAMPLE_FACTOR
    idxs = np.arange(0, len(filteredDft), step=DOWN_SAMPLE_FACTOR)
    dsDft = filteredDft[idxs]

    peaks, peakProps = sc.signal.find_peaks(
    dsDft,
    # threshold=0.01 * (np.max(dsDft) - np.min(dsDft)),
    prominence=0.01 * (np.max(dsDft) - np.min(dsDft)),
    width=(0, 5),
    wlen=3 # maybe change to 5
    )

    _peakWindows = []
    lenp = len(peaks)
    for i, p in enumerate(peaks):
        if i == 0:
            _peakWindows.append((0, int(p / 2)))
            p0 = int(p / 2)
        else:
            p0 = _peakWindows[-1][1]

        p1 = peaks[i + 1] if i < lenp - 1 else len(dsDft) - 1
        p1 = int((p1 - p) / 2 + p)

        _peakWindows.append((p0, p1))

        if i == lenp - 1:
            _peakWindows.append((_peakWindows[-1][1], len(dsDft) - 1))

    # ensure they index into original DFT
    _peakWindows = [
        (left * DOWN_SAMPLE_FACTOR, right * DOWN_SAMPLE_FACTOR) for left, right in _peakWindows
    ]

    if _peakWindows[-1][1] != len(halfDFT) - 1:
        _peakWindows[-1] = (_peakWindows[-1][0], len(halfDFT) - 1)

    peakWindows = np.array(_peakWindows)

    return peakWindows
    
# call:
# binEdges = getPeakBins(halfDFT, 8)

# # x_p = np.ones((1, len(_peakWindows)))
# -------- Bins Based on Peaks --------


In [ ]:
import numpy as np

# Based on LEMNA, using Gaussian Mixture Models to model decision boundary
# Returning all the models & clusters and the index of the model that contains 
# the original point (original point should be idx 0 of points and labels)

# This is NOT finished!!!
# It runs, but I am not sure it is doing what it is supposed to...

def predictWithGMM(points, labels, gmm, n_compontents):
    # gmm = BayesianGaussianMixture(n_components=n_components, covariance_type='full')
    data = np.concatenate((points, labels.reshape((1, -1)).T), axis=1)
    # for i in data:
    #     data[i].append(labels[i])

    # should the clusters be based on the points and labels or just points?
    gmm.fit(data) 

    models = [Ridge() for _ in range(n_components)]
    clusters = [[] for _ in range(n_components)]

    for i in range(n_components):
        clusters[i] = np.argwhere(gmm.predict(data) == i).T[0]
        # clusters[i] = np.argwhere(gmm.predict(X.T) == i).T[0]

    # for i in range(n_components):
    #     print("Cluster ", i, " dimensions: ", clusters[i].shape)
    #     # print(clusters[i])
    # print("Points dimensions: ", points.shape)

    for i, m in enumerate(models):
        if len(clusters[i]) != 0:
            m.fit(points[clusters[i]], labels[clusters[i]])
            # m.fit(X[0][clusters[i].reshape(-1,1)], X[1][clusters[i]])
    
    bestModel = 0

    for i in range(len(clusters)):
        if data[0] in clusters[i]:
            bestModel = i
            break

    return models, clusters, bestModel

In [ ]:
# For testing
# Testing retriving data stored

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.mixture import BayesianGaussianMixture
# from sklearn.linear_model import Ridge

# !pwd
# %cd /content/drive/MyDrive/Summer Research 2021/
# d = pd.read_csv('c_2k_Testing3.csv')


# z_pTest = []
# for i in range(len(d['z_p'])):
#     z_pTest.append(eval(d['z_p'][i]))

# z_pTest = np.array(z_pTest)
# print(z_pTest.shape)

# z_labsTest = []
# for i in range(len(d['z_labs'])):
#     z_labsTest.append(eval(d['z_labs'][i]))

# z_labsTest = np.array(z_labsTest)
# print(z_labsTest.shape)

# # for i in range(5):
# #     print(z_pTest[i][0][0])
# #     print(z_labsTest[i][0][0])

/content
/content/drive/MyDrive/Summer Research 2021
(5, 1, 2001, 25)
(5, 1, 2001)


In [ ]:
# Testing continued
# Testing Gaussian Mixture Model
# Also trying to see if I could use tqdm to get a runtime status

# import tqdm

# # GMM model
# n_components = 10

# for i in tqdm.trange(5, desc=f"Clustering Using Gaussian Mixture Model"):
#     gmm = BayesianGaussianMixture(n_components=n_components, covariance_type='full')
#     models, clusters, bestIdx = predictWithGMM(z_pTest[i][0], z_labsTest[i][0], gmm, n_components)
#     # print(models[bestIdx].coef_)


# # perturbations = tqdm.trange(self.nPert, desc=f"Perturbing Image")

Clustering Using Gaussian Mixture Model: 100%|██████████| 5/5 [00:07<00:00,  1.52s/it]


In [ ]:
!pwd
%cd /content/drive/MyDrive/Summer Research 2021/

import pandas as pd
import pickle
# import explainableai
import matplotlib.pyplot as plt
import random
import numpy as np
import scipy as sc
import scipy.signal
import automatic_speech_recognition as asr
from sklearn.linear_model import Ridge
from sklearn import metrics
import random
from pathlib import Path
from enum import Enum
from sklearn.mixture import BayesianGaussianMixture
import sys



class BinMethod(Enum):
    EVEN = 1
    OVERLAPPING = 2
    MEL = 3
    PEAK = 4

# class InterpModel(Enum):



N = 5         # Limiting the number of different audio samples we use
abc = 'c'     # hmm, I think we also define char = 'c' and use that later
# dictionary to story all the data 
dict = {
    #  'char' : [c for c in abc],
     'correct after perturbations %': [[] for _ in range(N)],
    #  'correct perturbations avg': [0 for _ in range(len(abc))],
     'audio name': ['' for _ in range(N)],
     'normalized': [[] for _ in range(N)],
     'origConf': [[] for _ in range(N)],
     'avgPertConf': [[] for _ in range(N)],
     'binMethod': [[] for _ in range(N)],
     'numBins': [[] for _ in range(N)],
     'binEdges': [[] for _ in range(N)],
     'overlapping': [[] for _ in range(N)],
     'sigma': [[] for _ in range(N)],
     'mse': [[] for _ in range(N)],
     'mse avg': [0 for _ in range(N)],
     'r2': [[] for _ in range(N)],
     'r2 avg': [0 for _ in range(N)],
     'z_p': [[] for _ in range(N)],
     'z_labs': [[] for _ in range(N)],
     'weights': [[] for _ in range(N)],
     'coefs': [[] for _ in range(N)]
    }
dict

/content/drive/MyDrive/Summer Research 2021
/content/drive/MyDrive/Summer Research 2021


{'audio name': ['', '', '', '', ''],
 'avgPertConf': [[], [], [], [], []],
 'binEdges': [[], [], [], [], []],
 'binMethod': [[], [], [], [], []],
 'coefs': [[], [], [], [], []],
 'correct after perturbations %': [[], [], [], [], []],
 'mse': [[], [], [], [], []],
 'mse avg': [0, 0, 0, 0, 0],
 'normalized': [[], [], [], [], []],
 'numBins': [[], [], [], [], []],
 'origConf': [[], [], [], [], []],
 'overlapping': [[], [], [], [], []],
 'r2': [[], [], [], [], []],
 'r2 avg': [0, 0, 0, 0, 0],
 'sigma': [[], [], [], [], []],
 'weights': [[], [], [], [], []],
 'z_labs': [[], [], [], [], []],
 'z_p': [[], [], [], [], []]}

In [ ]:
# %%
# loading pipeline outside of the for loops so it is not loaded 50 times
pipeline = asr.load('deepspeech2', lang='en')

# maxAmps = [5000, 10000, 15000, 20000, 25000]
# numBins = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]
# numBins = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
# numBins = [25, 40, 50, 100]
# sigmas = [0, .25, .5, .75, 1, np.sqrt(2), 1.5, 2, 2.5, 3, 5]

# for sig in sigmas:
for _ in range(1):
# for amp in maxAmps:
# for nBins in numBins:
    char = 'c'
    ALPHABET = ' abcdefghijklmnopqrstuvwxyz\'-'
    originalPercentage_ = []
    mse_ = []
    r2_ = []
    coefs_ = []
    audio_index = 0

    alignDir = Path('audio/alignments').expanduser().absolute()
    csvPaths = list(alignDir.rglob('*.csv'))
    df = pd.DataFrame()
    for csv in csvPaths:
        df = pd.concat((df, pd.read_csv(csv.as_posix())))
    metaData = df
    metaData['file_loc'] = metaData['file_loc'].apply(lambda x: Path(x).relative_to('speech_commands').as_posix())

    charTable = metaData.loc[metaData['letter'] == char]
    charTable = charTable[:N]  # Only using first N samples
    print(charTable)
    k = 0

    MAX_AMP = 20000
    print("Normalizing between", -MAX_AMP, "and", MAX_AMP)
    NUM_BINS = 25
    # BIN_METH could be EVEN, MEL, or PEAK
    # BIN_METH can be EVEN or OVERLAPPING
    BIN_METH = BinMethod.EVEN
    if (BIN_METH == BinMethod.OVERLAPPING):
        OVERLAPPING = True
    else:
        OVERLAPPING = False

    print("Using", NUM_BINS, end='')

    # if (BIN_METH == BinMethod.EVEN):
    #     print(" evenly spaced bins")
    # elif (BIN_METH == BinMethod.OVERLAPPING):
    #     print(" overlapping evenly spaced")

    if (BIN_METH == BinMethod.EVEN):
        print(" evenly spaced", end='')
    elif (BIN_METH == BinMethod.MEL):
        print(" mel spaced", end='')
    elif (BIN_METH == BinMethod.PEAK):
        print(" peak spaced", end='')

    if (OVERLAPPING):
        print(" overlapping bins")
    else:
        print(" bins")
 


    for _, row in charTable.iterrows():
        print('k is', k)
        charMeta = {'character': char, 'alphabetIdx': ALPHABET.index(char), 'alignIdx': row['alignment_index'], 'startIdx': row['st'], 'endIdx': row['ed'] }
        fileName = f'audio/speech_commands/{row["file_loc"]}'
        dict['audio name'][audio_index] = fileName
        audio_index += 1
        # print(charMeta)

        instance = asr.utils.read_audio(fileName)
        instance = np.pad(instance, 3000, 'reflect')

        # ----------------Normalize------------------
        # Want to normalize data between -MAX_AMP and MAX_AMP
        dict['normalized'][k].append(MAX_AMP)

        # print("dict['normalized']: ", dict['normalized'])

        # print("Instance: ", instance)
        maxAmp = instance[np.argmax(instance)]
        minAmp = instance[np.argmin(instance)]
        # print("maximum Amplitude: ", maxAmp)
        # print("minimum Amplitude: ", minAmp)
        
        maxMin = np.array([maxAmp, np.abs(minAmp)])
        maxMag = maxMin[np.argmax(maxMin)]
        # print("Maximum magnitude: ", maxMag)

        # plt.figure(figsize=(15, 9))
        # plt.subplot(1,1,1)
        # plt.title('Padded Audio')
        # plt.plot(instance)

        instance = instance/maxMag * MAX_AMP
        # print("Normalized instance: ", normInstance)
        # print("Normalized max: ", normInstance[np.argmax(normInstance)])
        # print("Normalized min: ", normInstance[np.argmin(normInstance)])
        # ----------------End Normalize------------------


        dft = sc.fft.fft(instance[charMeta['startIdx']:charMeta['endIdx']])
        halfDFT = dft[:int(np.floor(dft.shape[0] / 2.0) + 1)]



        # # Get bins
        # # for fixed bins, all bins start at 1 (not bothering with a threshold)
        # if (OVERLAPPING):
        #     nFirstBins = np.ceil(NUM_BINS/2).astype(int)
        #     # nSecondBins = nFirstBins - 1

        #     if (BIN_METH == BinMethod.EVEN):
        #         binEdges1 = getEvenBins(nFirstBins, len(halfDFT) - 1)
        #     elif (BIN_METH == BinMethod.MEL):
        #         binEdges1 = getMelBins(nFirstBins, len(halfDFT) - 1)
        #     elif (BIN_METH == BinMethod.PEAK):
        #         binEdges1 = getPeakBins(halfDFT, 8)

        #     # create bins halfway between other bins
        #     binEdges2 = []
        #     for i in range(len(binEdges1) - 1):
        #         binEdges2.append((((binEdges1[i][0] + binEdges1[i][1])/2).astype(int), ((binEdges1[i+1][0] + binEdges1[i+1][1])/2).astype(int)))
            
        #     # combine into an array of two tuples (each bin with its start and end)
        #     binEdges = []
        #     for i in range(len(binEdges1)):
        #         binEdges.append(binEdges1[i])
        #         if i < len(binEdges2):
        #             binEdges.append(binEdges2[i])

        # else:
        #     if (BIN_METH == BinMethod.EVEN):
        #         binEdges = getEvenBins(NUM_BINS, len(halfDFT) - 1)
        #     elif (BIN_METH == BinMethod.MEL):
        #         binEdges = getMelBins(NUM_BINS, len(halfDFT) - 1)
        #     elif (BIN_METH == BinMethod.PEAK):
        #         binEdges = getPeakBins(halfDFT, 8)

        if (BIN_METH == BinMethod.EVEN):
            binEdges = getEvenBins(NUM_BINS, len(halfDFT) - 1)
        elif (BIN_METH == BinMethod.OVERLAPPING):
            binEdges = getOverlappingEvenBins(NUM_BINS, len(halfDFT) - 1)


        # print()
        # print("Length of halfDFT: ", len(halfDFT))
        # # print("step: ", step)
        # print("Length of binEdges: ", len(binEdges))
        # print("binEdges: ", binEdges)
        # print()


        x_p = np.ones((1, len(binEdges)))


        # Define num perturbations and % max allowable perturbations
        numPerturb = 2000
        pMaxAllowablePerturb = 0.25

        # Max bins to perturn
        nMaxToPerturb = int(np.floor(pMaxAllowablePerturb * x_p.shape[1]))
        # Range of perturbations
        nToPerturbRange = list(np.arange(1, nMaxToPerturb + 1))
        # Create array of ones
        p = np.ones((numPerturb, x_p.shape[1]))
      
        for i in range(numPerturb):
            # perturbing all bins
            samplePop = list(np.arange(0, int(x_p.shape[1])))
            numIdxs = int(random.sample(nToPerturbRange, 1)[-1])
            idxs = random.sample(samplePop, numIdxs)
            idxs = np.array(idxs).astype(int)
            p[i, idxs] = 0

        z_p = np.concatenate((x_p.real, p), axis=0)
        interpInstance = z_p
        # Number of perturbed samples
        nPerturbed = interpInstance.shape[0]


        perturbedDFT = np.repeat([halfDFT], nPerturbed, axis=0)
        # Interpolate where bin is equal to 0
        for p in range(nPerturbed):
            for i in np.argwhere(interpInstance[p, :] == 0).flatten().tolist():
                # only remove the zeroed out peaks
                # left, right = peakWindows[i, :]
                left, right = binEdges[i]
                # left = binEdges[i]
                # right = binEdges[i+1]
                # perturbedDFT[p, left: right + 1] = np.interp(
                #     range(left, right + 1),
                #     [left, right],
                #     [perturbedDFT[p, left], perturbedDFT[p, right]]
                # )

                # perturb using noise
                # perturbedDFT[p, left: right + 1] = (np.random.uniform(-4, 4, (right+1-left,)) + 1j*np.random.uniform(-4, 4,(right+1-left,)))
                # perturb by setting intensity of frequencies to 0
                perturbedDFT[p, left: right + 1] = np.zeros(right+1-left)
                
                # if(p == 1):
                #   print("Perturbing frequencies ", left, " to ", right)
        
        # print()
        # print("interpInstance[1]: ", interpInstance[1])
        # print("perturbedDFT[1]: ", perturbedDFT[1])

        # plt.figure(figsize=(15, 9))
        # plt.subplot(1,1,1)
        # plt.title('Perturbed audio sample using 0s')
        # plt.plot(perturbedDFT[1])



        flipped = np.conj(np.flip(perturbedDFT, axis=1))
        flipped = flipped[:, 1:-1] if dft.shape[0] % 2 == 0 else flipped[:, :-1]
        perturbedDFT = np.concatenate((perturbedDFT, flipped), axis=1)

        batchAudio = np.zeros_like(perturbedDFT.real)

        for i in range(nPerturbed):
            batchAudio[i, :] = (sc.fft.ifft(perturbedDFT[i, :])).real

        output = np.repeat([instance], interpInstance.shape[0], axis=0)
        output[:, charMeta['startIdx']:charMeta['endIdx']] = batchAudio
        z = output

        # Extract spectrogram features
        features = pipeline._features_extractor(z)
        # Forward propagate 
        # TODO possibly use tqdm to give progress bar while running
        batchLogits = pipeline._model.predict(features)
        # Use softmax on the last layer
        charLogits = sc.special.softmax(batchLogits[:, charMeta['alignIdx'], :], axis=1)

        # Look at the probability at index 3 for c
        z_labs = charLogits[:, charMeta['alphabetIdx']]
        count = 0
        for c in charLogits:
            count += 1 if np.argmax(c) == charMeta['alphabetIdx'] else 0

        # magic happens here
        # originalPercentage_.append(count/charLogits.shape[0])


        def _normalize(x, _min, _max):
            x = x.astype(np.int32)
            output = (x - _min) / (_max - _min)
            return output

        def distanceL2(x, z, charMeta):
            dMin, dMax = np.iinfo(np.int16).min, np.iinfo(np.int16).max
            xnorm = _normalize(x[charMeta['startIdx']:charMeta['endIdx']], dMin, dMax)
            znorm = _normalize(z[charMeta['startIdx']:charMeta['endIdx']], dMin, dMax)

            return np.linalg.norm(xnorm - znorm)



        d_p = np.apply_along_axis(lambda a: distanceL2(instance, a, charMeta), axis=1, arr=z)
        d = d_p
        sigma = np.sqrt(np.std(d))
        # if (sig == 0):
        #     sigma = np.sqrt(np.std(d))
        # else:
        #     sigma = sig
        weights = np.exp(-(d**2)/ sigma ** 2.0)

        # print()
        # # print("distances: ", d)
        # # print("sigma: ", sigma)
        # # print("weights: ", weights)
        # print("sigma: ", sigma)
        # print("sqrt of std of distances:", np.sqrt(np.std(d)))
        # print("max distance:", d[np.argmax(d)])
        # print("min weight:", weights[np.argmin(weights)])
        # print("average distance:", np.average(d))
        # print("average weight:", np.average(weights))
        # print()

        # GMM model (LEMNA)
        n_components = 10
        gmm = BayesianGaussianMixture(n_components=n_components, covariance_type='full')
        # model = predictWithGMM(z_p, z_labs, gmm, n_components)
        models, clusters, bestIdx = predictWithGMM(z_p, z_labs, gmm, n_components)
        model = models[bestIdx]
        y_pred = model.predict(z_p)
        r2 = model.score(z_p[clusters[bestIdx]], z_labs[clusters[bestIdx]])
        mse = metrics.mean_squared_error(z_labs[clusters[bestIdx]], y_pred[clusters[bestIdx]])
        coefs = model.coef_

        # # Regular model (LIME)
        # model = Ridge()
        # model.fit(z_p, z_labs, sample_weight = weights)
        # y_pred = model.predict(z_p)
        # r2 = model.score(z_p, z_labs, sample_weight = weights)
        # mse = metrics.mean_squared_error(z_labs, y_pred, sample_weight=weights)
        # coefs = model.coef_
        

        # print("r2:", r2)
        # print()


        # This is to store z_p without elipsis
        np.set_printoptions(threshold=sys.maxsize)
        # Got from: https://stackoverflow.com/questions/25375260/pandas-to-csv-always-substitute-long-numpy-ndarray-with-ellipsis

        print(char, abc.index(char), 'type:', type(abc.index(char)))
        dict['binMethod'][k].append(BIN_METH.name)
        dict['numBins'][k].append(len(binEdges))
        dict['binEdges'][k].append(binEdges)
        dict['overlapping'][k].append(OVERLAPPING)
        dict['sigma'][k].append(sigma)
        dict['correct after perturbations %'][k].append(count/charLogits.shape[0])
        dict['mse'][k].append(mse)
        dict['r2'][k].append(r2)
        dict['z_p'][k].append(z_p.tolist())         
        dict['z_labs'][k].append(z_labs.tolist()) 
        dict['weights'][k].append(weights.tolist())   
        dict['coefs'][k].append(coefs.tolist())       
        dict['origConf'][k].append(z_labs[0])
        dict['avgPertConf'][k].append(np.average(z_labs[1:]))
        k+=1

    # dict['correct perturbations avg'][abc.index(char)] = sum(dict['correct after perturbations %'][abc.index(char)]) / len(dict['correct after perturbations %'][abc.index(char)])
    # dict['mse avg'][abc.index(char)] = sum(dict['mse'][abc.index(char)]) / len(dict['mse'][abc.index(char)])
    # dict['r2 avg'][abc.index(char)] = sum(dict['r2'][abc.index(char)]) / len(dict['r2'][abc.index(char)])
    
    #     break
    # break

for i in range(5): # I think this should be the N = 5 from earlier
    dict['mse avg'][i] = sum(dict['mse'][i]) / len(dict['mse'][i])
    dict['r2 avg'][i] = sum(dict['r2'][i]) / len(dict['r2'][i])
df=pd.DataFrame(dict)
# TODO: Change it so you don't have to manually change the name of the file every time
# and if you forget to, it doesn't override your last thing of data.
df.to_csv('c_2k_TestingGaussian3.csv')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
                    file_loc   gt transcript  ... alignment_index    st     ed
0  cat/9f869f70_nohash_0.wav  cat        cat  ...              33  8160  13279
0  cat/f618568f_nohash_0.wav  cat        cat  ...              29  6880  11999
0  cat/e9a76b2f_nohash_0.wav  cat        cat  ...              27  6240  11359
0  cat/ba676390_nohash_0.wav  cat        cat  ...              33  8160  13279
0  cat/db7c95b0_nohash_0.wav  cat        cat  ...              20  4000   9119

[5 rows x 7 columns]
Normalizing between -20000 and 20000
Using 25 evenly spaced bins
k is 0


TypeError: ignored

In [ ]:
# print(binEdges)
# print("correct after perturbations %: ", dict['correct after perturbations %'])
# print("audio name: ", dict['audio name'])
# print("normalized: ", dict['normalized'])
# print("origConf: ", dict['origConf'])
# print("avgPertConf: ", dict['avgPertConf'])
# print("binEdges: ", dict['binEdges'])
# print("mse: ", dict['mse'])
# print("mse avg: ", dict['mse avg'])
# print("r2: ", dict['r2'])
# print("r2 avg: ", dict['r2 avg'])
# print("coefs: ", dict['coefs'])

# dict['binEdges'][0] = list(binEdges)
# print("binEdges: ", dict['binEdges'])
# print("binEdges[0]: ", dict['binEdges'][0])



# df=pd.DataFrame(dict)
# df.to_csv('c_2k_pert0_evenBins.csv')

In [ ]:
print("distances:\n", d[0:30])
print("weights:\n", weights[0:30])
print("max distance:", d[np.argmax(d)])
print("min distance:", d[np.argmin(d[1:])])
print("max weight:", weights[np.argmax(weights[1:])])
print("min weight:", weights[np.argmin(weights)])

In [ ]:
print("sqrt of std of distances:", np.sqrt(np.std(d)))
print("max distance:", d[np.argmax(d)])
print("min weight:", weights[np.argmin(weights)])
print("average distance:", np.average(d))
print("average weight:", np.average(weights))